In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
from google.colab import drive

drive.mount('/content/gdriv')
PATH ='/content/gdriv/My Drive/LG/data'

Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


In [ ]:
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    minute  = int(x[10] + '0')
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour, minute)

In [ ]:
train_err  = pd.read_csv(PATH+'/train_err_data.csv')
train_qual  = pd.read_csv(PATH+'/train_quality_data.csv')
train_prob  = pd.read_csv(PATH+'/train_problem_data.csv')

In [ ]:
train_err['time'] = train_err['time'].apply(make_datetime).dt.strftime('%Y%m%d%H%M%S')


In [ ]:
train_err

user_id            time model_nm       fwver  errtype errcode
0           10000  20201101025000  model_3  05.15.2138       15       1
1           10000  20201101030000  model_3  05.15.2138       12       1
2           10000  20201101030000  model_3  05.15.2138       11       1
3           10000  20201101050000  model_3  05.15.2138       16       1
4           10000  20201101050000  model_3  05.15.2138        4       0
...           ...             ...      ...         ...      ...     ...
16554658    24999  20201130163000  model_3  05.15.2138       15       1
16554659    24999  20201130172000  model_3  05.15.2138       16       1
16554660    24999  20201130172000  model_3  05.15.2138        4       0
16554661    24999  20201130172000  model_3  05.15.2138        4       0
16554662    24999  20201130210000  model_3  05.15.2138       15       1

[16554663 rows x 6 columns]

애초에 저기 err 데이터 형태를 갯수 형태로 바꾸고, 그후에 시점별 sum

In [ ]:
train_err['idx']=train_err['time'].astype(str)+train_err['user_id'].astype(str)

In [ ]:
train_err

user_id            time  ... errcode                  idx
0           10000  20201101025000  ...       1  2020110102500010000
1           10000  20201101030000  ...       1  2020110103000010000
2           10000  20201101030000  ...       1  2020110103000010000
3           10000  20201101050000  ...       1  2020110105000010000
4           10000  20201101050000  ...       0  2020110105000010000
...           ...             ...  ...     ...                  ...
16554658    24999  20201130163000  ...       1  2020113016300024999
16554659    24999  20201130172000  ...       1  2020113017200024999
16554660    24999  20201130172000  ...       0  2020113017200024999
16554661    24999  20201130172000  ...       0  2020113017200024999
16554662    24999  20201130210000  ...       1  2020113021000024999

[16554663 rows x 7 columns]

In [ ]:
idx=train_err['idx']

In [ ]:
new_idx=[]
a=0
for i in range(len(idx)):
  if i==0:
    new_idx.append(0)
    a=idx[0]
  else:
    if idx[i]==a:
      new_idx.append(new_idx[i-1])
    else: 
      a=idx[i]
      new_idx.append(new_idx[i-1]+1)    



In [ ]:
train_err['new_idx']=new_idx

In [ ]:
train_err

user_id            time  ...                  idx  new_idx
0           10000  20201101025000  ...  2020110102500010000        0
1           10000  20201101030000  ...  2020110103000010000        1
2           10000  20201101030000  ...  2020110103000010000        1
3           10000  20201101050000  ...  2020110105000010000        2
4           10000  20201101050000  ...  2020110105000010000        2
...           ...             ...  ...                  ...      ...
16554658    24999  20201130163000  ...  2020113016300024999  3717267
16554659    24999  20201130172000  ...  2020113017200024999  3717268
16554660    24999  20201130172000  ...  2020113017200024999  3717268
16554661    24999  20201130172000  ...  2020113017200024999  3717268
16554662    24999  20201130210000  ...  2020113021000024999  3717269

[16554663 rows x 8 columns]

In [ ]:
#train_err['errtype']=train_err['errtype'].astype(str)+train_err['errcode'].astype(str)

In [ ]:
#train_err

In [ ]:
print(np.sort(train_err.errtype.unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [ ]:
train_err.dtypes

user_id      int64
time        object
model_nm    object
fwver       object
errtype      int64
errcode     object
idx         object
new_idx      int64
dtype: object

In [ ]:
#train_err['errtype']=train_err['errtype'].astype(int)

In [ ]:
train_user_id_max = 3717270
train_user_id_min = 0
train_user_number = 3717270

## baseline코드와 같은 셀

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['new_idx','errtype']].values
print(id_error)
error = np.zeros((train_user_number,42))  #15000,42의 어레이 형테를 0벡터로 만들고

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx,err - 1] += 1

error.shape
error

[[      0      15]
 [      1      12]
 [      1      11]
 ...
 [3717268       4]
 [3717268       4]
 [3717269      15]]


100%|██████████| 16554663/16554663 [00:43<00:00, 381283.82it/s]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
error.shape

(3717270, 42)

In [ ]:
train_prob

user_id            time
0       19224  20201102200000
1       23664  20201116140000
2       15166  20201114130000
3       12590  20201108210000
4       15932  20201103210000
...       ...             ...
5424    20167  20201125120000
5425    16270  20201110120000
5426    19114  20201106230000
5427    21505  20201104110000
5428    18822  20201102120000

[5429 rows x 2 columns]

In [ ]:
problem = np.zeros(3717270)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()] = 1 
problem.shape

(3717270,)

In [ ]:
train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

(3717270, 42)
(3717270,)


In [ ]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.1
# 파라미터 설정
params =      {
#                'learning_rate' : 0.0001,
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train( 
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.631238	valid_0's pr_auc: 0.00669312
[40]	valid_0's auc: 0.633043	valid_0's pr_auc: 0.00652149
[60]	valid_0's auc: 0.630964	valid_0's pr_auc: 0.00634297
[80]	valid_0's auc: 0.632582	valid_0's pr_auc: 0.00649591
[100]	valid_0's auc: 0.63071	valid_0's pr_auc: 0.00646651
Early stopping, best iteration is:
[1]	valid_0's auc: 0.570423	valid_0's pr_auc: 0.00779931
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.611566	valid_0's pr_auc: 0.00574833
[40]	valid_0's auc: 0.615954	valid_0's pr_auc: 0.00545656
[60]	valid_0's auc: 0.613742	valid_0's pr_auc: 0.00542299
[80]	valid_0's auc: 0.614295	valid_0's pr_auc: 0.00541745
[100]	valid_0's auc: 0.616617	valid_0's pr_auc: 0.0047645
Early stopping, best iteration is:
[1]	valid_0's auc: 0.552945	valid_0's pr_auc: 0.00849012
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.608627	valid_0's pr_auc: 

In [ ]:
print(np.mean(auc_scores))

0.559985328116665
